In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers, callbacks
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Flatten, Dense, Dropout
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras import backend as K

Using TensorFlow backend.


In [2]:
train_idg = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True
    )
val_idg = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True
    )
test_idg = ImageDataGenerator()

In [3]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
vgg19_conv = VGG19(weights='imagenet', include_top=False)

# Freeze the layers except the last 4 layers
for layer in vgg19_conv.layers[:-4]:
    layer.trainable = False
    
# Check the trainable status of the individual layers
for layer in vgg19_conv.layers:
    print(layer, layer.trainable)

(<keras.engine.topology.InputLayer object at 0x7fba9f86af90>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9f8d3ad0>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fbb24fdcd90>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7fba9f87c150>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9f8d3910>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9f89fad0>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7fba9f047f10>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9f05af10>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9efe9c50>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9f008e90>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9f01a8d0>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7fba9efb7790>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9efd4ad0>, False)
(<keras.layers.convolutional.Conv2D object at 0x7fba9ef61a10>, False)
(<keras.layers.convol

In [ ]:
#Create your own input format (here 3x200x200)
input = Input(shape=(91,109,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

#Create your own model 
my_model = Model(input=input, output=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

In [8]:
#Then training with your data ! 
# Compile the model
# Model Loss function and Optimizer method
from keras.callbacks import CSVLogger,ModelCheckpoint, EarlyStopping

DEST = '/media/avell/Novo volume/GM_CORONAL/'
img_width, img_height = 91, 91
nb_train_samples = 4125
nb_validation_samples = 466 
batch_size = 16
epochs = 50

for i in range(10):

    fold = DEST+'fold_{}'.format(i)
    train_g = train_idg.flow_from_directory(
        fold+'/train',
        target_size = (img_height, img_width),
        batch_size= 16,
        class_mode='binary')
    val_g   = val_idg.flow_from_directory(
        fold+'/validation',
        target_size = (img_height, img_width),
        batch_size= 16,
        class_mode='binary')

    #Get back the convolutional part of a VGG network trained on ImageNet
    model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))

    # Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
    for layer in model.layers[:5]:
        layer.trainable = False

    #Adding custom Layers 
    x = model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(1, activation="sigmoid")(x)

    # creating the final model 
    model_final = Model(input = model.input, output = predictions)
    
    # compile the model 
    model_final.compile(optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
                                     loss='binary_crossentropy',metrics=['accuracy'])

    # Settting Callbacks
    checkpoint = ModelCheckpoint(fold+"/vgg19_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
    csv_logger = CSVLogger(fold+'/log.csv', append=True, separator=';')

    callb_l = [checkpoint,csv_logger,early]
    # Train the model
    model_final.fit_generator(
          train_g,
          samples_per_epoch = nb_train_samples,
          epochs = epochs,
          nb_val_samples = nb_validation_samples,
          validation_data=val_g,
          verbose=1,
          callbacks=callb_l)

    # Save the model
    K.clear_session()

Found 152 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


/home/avell/anaconda3/envs/p3env/lib/python2.7/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
/home/avell/anaconda3/envs/p3env/lib/python2.7/site-packages/ipykernel_launcher.py:63: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/avell/anaconda3/envs/p3env/lib/python2.7/site-packages/ipykernel_launcher.py:63: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, validation_data=<keras.pre..., steps_per_epoch=257, epochs=50, callbacks=[<keras.ca..., validation_steps=466)`


Epoch 1/50
257/257 [==============================] - 42s 162ms/step - loss: 0.7038 - acc: 0.5214 - val_loss: 0.6931 - val_acc: 0.5157

Epoch 00001: val_acc improved from -inf to 0.51574, saving model to /media/avell/Novo volume/GM_CORONAL/fold_0/vgg19_1.h5
Epoch 2/50
257/257 [==============================] - 41s 159ms/step - loss: 0.6666 - acc: 0.5795 - val_loss: 0.6805 - val_acc: 0.5508

Epoch 00002: val_acc improved from 0.51574 to 0.55079, saving model to /media/avell/Novo volume/GM_CORONAL/fold_0/vgg19_1.h5
Epoch 3/50
257/257 [==============================] - 42s 162ms/step - loss: 0.6150 - acc: 0.6620 - val_loss: 0.7076 - val_acc: 0.5439

Epoch 00003: val_acc did not improve
Epoch 4/50
257/257 [==============================] - 41s 160ms/step - loss: 0.5630 - acc: 0.7050 - val_loss: 0.7394 - val_acc: 0.6435

Epoch 00004: val_acc improved from 0.55079 to 0.64354, saving model to /media/avell/Novo volume/GM_CORONAL/fold_0/vgg19_1.h5
Epoch 5/50
257/257 [===========================

KeyboardInterrupt: 

In [10]:
from keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import csv

DEST = '/media/avell/Novo volume/GM_CORONAL/'

for i in range(10):
    K.clear_session()
    
    fold = DEST+'fold_{}'.format(i)
    test_g = test_idg.flow_from_directory(
        fold+'/test',
        target_size=(91,91),
        batch_size= 10,
        class_mode='binary')

    my_model = load_model(filepath=fold+'/vgg19_1.h5')

    (eval_loss, eval_acc) = my_model.evaluate_generator(generator=test_g,steps=1)
    print('\nEvaluation accuracy: {:.2f}'.format(eval_acc))

    pred = my_model.predict_generator(test_g)

    pred[pred >= 0.5] = 1
    pred[pred != 1 ] = 0
    
    pr,rl,f1,sp = precision_recall_fscore_support(pred, test_g.classes)
    acc = accuracy_score(pred, test_g.classes)
    tn, fp, fn, tp = confusion_matrix(pred, test_g.classes).ravel()
    specificity = 1.0*tn / (tn+fp)
    
    with open(fold+'/fold_{}_metrics.csv'.format(i),'wb') as csvFile:
        filewriter = csv.writer(csvFile)
        filewriter.writerow(['class','precision','recall','fscore','support','accuracy','specificity'])
        filewriter.writerow([0,pr[0],rl[0],f1[0],sp[0],'-','-'])
        filewriter.writerow([1,pr[1],rl[1],f1[1],sp[1],'-','-'])
        filewriter.writerow(['-','-','-','-','-',acc,specificity])
    csvFile.close()
    
    np.save(fold+'/fold_{}_pred_labels.npy'.format(i),pred)

Found 18 images belonging to 2 classes.

Evaluation accuracy: 0.70
Found 18 images belonging to 2 classes.

Evaluation accuracy: 0.40
Found 18 images belonging to 2 classes.

Evaluation accuracy: 0.60
Found 17 images belonging to 2 classes.


IOError: Unable to open file (unable to open file: name = '/media/avell/Novo volume/GM_CORONAL/fold_3/vgg19_1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

#### SuperLearner features creation

In [ ]:
DEST = '/media/avell/Novo volume/OASIS_CV_axial/'
EXT = '_a.jpg'
FIG_SHAPE = (208,176)

for i in range(10):
    ldst = DEST+'fold_{}/train/'.format(i)
    k_labels = []
    for j in range(10):
        fold = ldst+'fold_{}'.format(j)
        train_g = train_idg.flow_from_directory(
            fold+'/train',
            target_size=FIG_SHAPE,
            batch_size= 10,
            class_mode='binary')
        val_g   = val_idg.flow_from_directory(
            fold+'/validation',
            target_size=FIG_SHAPE,
            batch_size= 10,
            class_mode='binary')
        test_g = test_idg.flow_from_directory(
            fold+'/test',
            target_size=FIG_SHAPE,
            batch_size= 10,
            class_mode='binary')

        #Get back the convolutional part of a VGG network trained on ImageNet
        vgg16_conv = VGG16(weights='imagenet', include_top=False)

        # Freeze the layers except the last 4 layers
        for layer in vgg16_conv.layers[:-4]:
            layer.trainable = False

        #Create your own input format (here 3x200x200)
        input = Input(shape=(FIG_SHAPE[0],FIG_SHAPE[1],3),name = 'image_input')

        #Use the generated model 
        output_vgg16_conv = vgg16_conv(input)

        #Add the fully-connected layers 
        x = Flatten(name='flatten')(output_vgg16_conv)
        x = Dense(4096, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(1, activation='sigmoid')(x)

        #Create your own model 
        my_model = Model(input=input, output=x)

        #In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
        my_model.summary()

        compile = my_model.compile(optimizer=optimizers.RMSprop(lr=1e-5), loss='binary_crossentropy',
                               metrics=['accuracy'])

        # Settting Callbacks
        reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3,verbose=1, cooldown=2)

        callb_l = [reduce_lr]
        # Train the model
        my_model.fit_generator(
              train_g,
              steps_per_epoch=train_g.samples/train_g.batch_size ,
              epochs=40,
              validation_data=val_g,
              validation_steps=val_g.samples/val_g.batch_size,
              verbose=1,
              callbacks=callb_l)
        
        my_model.evaluate_generator(generator=test_g,steps=1)
        pred = my_model.predict_generator(test_g)

        K.clear_session()
        
        pred[pred >= 0.5] = 1
        pred[pred != 1 ] = 0
        
        k_labels.append(pred.ravel())
    np.save('features_'+'fold_{}'.format(i),np.array(k_labels))

In [ ]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix,classification_report

K.clear_session()

my_model = load_model(filepath='TL_OASIS_ax.h5')
#print(my_model.summary())

(eval_loss, eval_acc) = my_model.evaluate_generator(generator=test_g,steps=1)
print('\nEvaluation accuracy: {:.2f}'.format(eval_acc))

pred = my_model.predict_generator(test_g)

pred[pred >= 0.5] = 1
pred[pred != 1 ] = 0

print(classification_report(test_g.classes, pred.ravel()))
print(np.reshape(pred,(1,len(pred))), test_g.classes)

In [ ]:
pred_train = my_model.predict_generator(train_g)
pred_val   = my_model.predict_generator(val_g)
pred_test = my_model.predict_generator(test_g)

In [ ]:
arrayTrain = np.vstack((pred_train,pred_val))
trainY = np.hstack((train_g.classes,val_g.classes))

In [ ]:
np.save('df_train_x.npy',arrayTrain)
np.save('df_test_x.npy',pred_test)
np.save('df_train_y.npy',trainY)
np.save('df_test_y.npy',test_g.classes)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report

svc = SVC(C=100,gamma=2e-4)
svc.fit(arrayTrain,trainY)
pred_ar = svc.predict(pred_test)
print(trainY)
print(classification_report(pred_ar,test_g.classes))